In [43]:
import numpy as np
import matplotlib.pyplot as plt


In [48]:
def logprior(v,theta):
    p = -np.inf
    if v > 0 and v<100 and theta<np.pi/2 and theta>0:
        p = 0.0
    return p

def modelo(v, theta):
    g = 9.8
    return v**2 * np.sin(2*theta)/g

def loglikelihood(d_obs, sigma_d, v, t):
    d = d_obs -  modelo(v, t)
    d = d/sigma_d
    d = -0.5 * np.sum(d**2)
    return d

d_obs = np.array([880, 795, 782, 976, 178])
sigma_d = 5.0

In [54]:
N = 5000
lista_v = [45]
lista_theta = [np.pi/3]
sigma_v=10
sigma_theta = 1
for j in range(len(d_obs)):
    d_real = d_obs[j]
    logposterior = [loglikelihood(d_real,sigma_d,lista_v[-1],lista_theta[-1]) + logprior(lista_v[-1],lista_theta[-1])]
    for i in range(1,N):
        propuesta_v  = lista_v[i-1] + np.random.normal(loc=0.0, scale=sigma_v)
        propuesta_theta  = lista_theta[i-1] + np.random.normal(loc=0.0, scale=sigma_theta)

        logposterior_viejo = loglikelihood(d_real,sigma_d,lista_v[i-1],lista_theta[i-1]) + logprior(lista_v[i-1],lista_theta[i-1])
        logposterior_nuevo = loglikelihood(d_real,sigma_d,propuesta_v,propuesta_theta) + logprior(propuesta_v,propuesta_theta)
                                                                                               
        r = min(1,np.exp(logposterior_nuevo-logposterior_viejo))
        alpha = np.random.random()
        if(alpha<r):
            lista_v.append(propuesta_v)
            lista_theta.append(propuesta_theta)
        else:
            lista_v.append(lista_v[i-1])
            lista_theta.append(lista_theta[i-1])
            
lista_v = np.array(lista_v)
lista_theta = np.array(lista_theta)

#print(lista_v)
plt.plot(lista_)
#plt.hist(lista_v, bins=100, density=True)
#plt.savefig('velocidad.pdf')
#plt.errorbar(x_obs,z_obs, yerr=sigma_z, fmt='o')
#plt.plot(new_x, y_model)


/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()
